## Q1. Getting the embeddings model

In [8]:
from sentence_transformers import SentenceTransformer
model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

In [9]:
user_question = "I just discovered the course. Can I still join it?"
embedding_model.encode(user_question)

array([ 7.82226324e-02, -4.01311293e-02,  3.86136100e-02, -1.78962495e-04,
        8.92346650e-02, -5.04591130e-02, -1.05026821e-02,  3.71055938e-02,
       -4.18714136e-02,  3.48085277e-02, -1.20702265e-02, -2.36942414e-02,
        3.87899876e-02,  1.60988141e-02,  3.50747034e-02,  3.04755941e-03,
        5.79672270e-02, -4.10627499e-02, -3.41552868e-02, -2.56396476e-02,
       -3.55264395e-02,  1.42907910e-02, -1.62799843e-02,  3.21446396e-02,
       -4.66897450e-02,  7.89185986e-02,  4.90160957e-02,  1.56760849e-02,
       -1.69110298e-02,  2.26482041e-02,  5.60206249e-02, -3.98361534e-02,
        6.77410141e-02, -1.20210061e-02,  1.12624047e-03, -1.94394924e-02,
       -2.65951231e-02,  1.06177963e-02,  1.69687681e-02,  1.13488007e-02,
       -2.97063068e-02,  5.25258482e-02, -1.41453156e-02,  4.61699590e-02,
        1.17066344e-02, -2.38052998e-02, -6.32557869e-02, -1.92041937e-02,
       -7.10595725e-03,  3.24167944e-02,  2.49618031e-02, -5.27500687e-03,
        2.01149434e-02, -

so, first value of the resulting vector is `7.82226324e-02`, i.e., ~ $0.07$.

#### 1.2 Prepare the documents

In [28]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [51]:
# looping over the documents to filter the documents and create a list of documents only with the questions for "machine-learning-zoomcamp". we need every existing info of the documents containing the questions for "machine-learning-zoomcamp" to be able to find the most similar question to the user question. let's filter out every document that is not related to "machine-learning-zoomcamp" and keep the remaining documents. 'documents' is a dictionary with the following keys: 'id', 'course', 'module', 'lesson', 'content'. 'course' is a list with one element, 'machine-learning-zoomcamp'. 'module' is a string with the format 'module-<number>'. 'lesson' is a string with the format 'lesson-<number>'. 'content' is a string with the content of the document.

ml_zoomcamp_documents = []
for doc in documents:
    if doc['course'] == 'machine-learning-zoomcamp':
        ml_zoomcamp_documents.append(doc)

len(ml_zoomcamp_documents)

375

In [55]:
ml_zoomcamp_documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872'}

## Creating the embeddings
Now for each document, we will create an embedding for both question and answer fields.

We want to put all of them into a single matrix `X`:

- Create a list `embeddings` 
- Iterate over each document 
- `qa_text = f'{question} {text}'`
- compute the embedding for `qa_text`, append to `embeddings`
- At the end, let `X = np.array(embeddings)` (`import numpy as np`) 

What's the shape of X? (`X.shape`). Include the parantheses. 

In [56]:
# list called 'embeddings'
# Now for each document, we will create an embedding for both question and answer fields.

# We want to put all of them into a single matrix `X`:

# - Create a list `embeddings` 
# - Iterate over each document 
# - qa_text = f'{question} {text}'
# - compute the embedding for `qa_text`, append to `embeddings`
# - At the end, let `X = np.array(embeddings)` (`import numpy as np`) 

import numpy as np

embeddings = []
for doc in ml_zoomcamp_documents:
    qa_text = f'{doc["question"]} {doc["text"]}'
    embeddings.append(embedding_model.encode(qa_text))

X = np.array(embeddings)

In [57]:
X.shape

(375, 768)

Shape of X: (375, 768)

## Q3. Search
We have the embeddings and the query vector. Now let's compute the 
cosine similarity between the vector from Q1 (let's call it `v`) and the matrix from Q2. 

The vectors returned from the embedding model are already
normalized (you can check it by computing a dot product of a vector
with itself - it should return something very close to 1.0). This means that in order
to compute the coside similarity, it's sufficient to 
multiply the matrix `X` by the vector `v`:


```python
scores = X.dot(v)
```

What's the highest score in the results?

- 65.0 
- 6.5
- 0.65
- 0.065

In [62]:
# We have the embeddings and the query vector. Now let's compute the 
# cosine similarity between the vector from Q1 (let's call it `v`) and the matrix from Q2. 

# The vectors returned from the embedding model are already
# normalized (you can check it by computing a dot product of a vector
# with itself - it should return something very close to 1.0). This means that in order
# to compute the coside similarity, it's sufficient to 
# multiply the matrix `X` by the vector `v`:


# ```python
# scores = X.dot(v)
# ```

# calculating the cosine similarity between the user question and the documents
v = embedding_model.encode(user_question)
scores = np.dot(X, v)
scores

array([ 2.89217472e-01,  4.35050964e-01,  2.20572710e-01,  1.28085911e-01,
        3.28754783e-01,  4.58999842e-01,  4.48931158e-01,  2.70170718e-01,
        2.13975042e-01,  3.39745700e-01,  1.63339168e-01,  3.15918058e-01,
        1.53620597e-02,  4.14240599e-01,  6.50657535e-01,  4.12786633e-01,
        2.02390462e-01,  4.71707582e-01,  3.74821484e-01,  3.53319287e-01,
        4.50558513e-01,  1.85430795e-01,  3.45527172e-01,  8.22948888e-02,
       -2.19077207e-02, -4.64249849e-02,  8.60690624e-02,  9.87197086e-02,
       -2.14188509e-02,  1.28752766e-02,  2.54870132e-02,  3.66865620e-02,
        4.56339158e-02, -2.05278452e-02,  1.82661451e-02,  3.39788869e-02,
        1.66492118e-03,  1.03211123e-02,  3.40170003e-02, -2.65231114e-02,
       -4.11910079e-02, -4.97983247e-02, -6.20347112e-02, -7.63558224e-03,
        4.61617485e-02,  6.95947558e-03,  4.77613024e-02, -9.97874886e-04,
       -5.60125150e-02, -5.96207082e-02,  2.05209255e-01, -1.16968535e-01,
       -2.34159492e-02, -

In [65]:
# highest score
highest_score = scores.max()
highest_score

0.65065753

Highest score = 0.65

## Vector search

We can now compute the similarity between a query vector and all the embeddings.

Let's use this to implement our own vector search

```python
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)
```

If you don't understand how the `search` function work:

* Ask ChatGTP or any other LLM of your choice to explain the code
* Check our pre-course workshop about implementing a search engine [here](https://github.com/alexeygrigorev/build-your-own-search-engine)

(Note: you can replace `argsort` with `argpartition` to make it a lot faster)

In [66]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'You can find the latest and up-to-date deadlines here: https://docs.google.com/spreadsheets/d/e/2PACX-1vQACMLuutV5rvXg5qICuJGL-yZqIV0FBD84CxPdC5eZHf8TfzB-CJT_3Mo7U7oGVTXmSihPgQxuuoku/pubhtml\nAlso, take note of Announcements from @Au-Tomator for any extensions or other news. Or, the form may also show the updated deadline, if Instructor(s) has updated it.',
  'section': 'General course-related questions',
  'question': 'Homework - What are homework and project deadlines?',
  'course': 'data-engineering-zoomcamp',
  'id': 'a1daf537'},
 {'text': 'After you submit your homework it will be graded based on the amount of questions in a particular homework. You can see how many points you have right on the page of the homework up top. Additionally in the leaderboard you will find the sum of all points you’ve earned - points for Homeworks, FAQs and Learning in Public. If homework is clear, others work as follows: if you submit something to FAQ, you get one point, for each learning i

## Q4. Hit-rate for our search engine

Let's evaluate the performance of our own search engine. We will
use the hitrate metric for evaluation.

First, load the ground truth dataset:

```python
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')
```

Now use the code from the module to calculate the hitrate of
`VectorSearchEngine` with `num_results=5`.

What did you get?

* 0.93
* 0.73
* 0.53
* 0.33

In [107]:
import pandas as pd
from tqdm import tqdm

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

v_q = embedding_model.encode(user_question)

# def elastic_search_knn(field, vector, course):
#     knn = {
#         fi
    # }

In [110]:

def hit_rate(relevance_total):
    count = 0
    for relevance in relevance_total:
        if True in relevance:
            count += 1

    return count / len(relevance_total)

# # Now use the code from the module to calculate the hitrate of
# # `VectorSearchEngine` with `num_results=5`.

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [doc['id'] == doc_id for doc in results]
        relevance_total.append(relevance)

    return hit_rate(relevance_total)

  0%|          | 0/1830 [00:00<?, ?it/s]


TypeError: 'list' object is not callable